Sveučilište u Zagrebu  
Fakultet elektrotehnike i računarstva  
  
## Uvod u znanost o podacima 2023/2024 
https://www.fer.unizg.hr/predmet/uuzop

------------------------------

### Projekt: Early-Stage Alzheimer's Disease Prediction Using Machine Learning Models

*https://www.frontiersin.org/articles/10.3389/fpubh.2022.853294/full*

**Veronika Žunar, 0036525058**

------------------------------


### Vizualizacija podataka
### Analiza korelacije i zavisnosti
### Napredna analiza

In [1]:
import numpy as np
import pandas as pd

### 1. Učitavanje podataka

In [9]:
df = pd.read_csv(".\dataset\oasis_longitudinal.csv")

In [10]:
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [41]:
cols = df.columns
desc = ["Identification","", "", "","", "Gender (M if Male, F if Female)", "Handedness", "Age in years", "Years of education", "Socio Economic Status", "Mini Mental State Examination", "Clinical Dementia Rating", "Estimated Total Intracranial Volume", "Normalize Whole Brain Volume", "Atlas Scaling Factor"]
types = df.dtypes

for col, d, typ in zip(cols, desc, types):
    print(f'\033[1m{col}\033[0m - {d} \033[1m[{typ}]\033[0m')

Subject ID - Identification [object]
MRI ID -  [object]
Group -  [object]
Visit -  [int64]
MR Delay -  [int64]
M/F - Gender (M if Male, F if Female) [object]
Hand - Handedness [object]
Age - Age in years [int64]
EDUC - Years of education [int64]
SES - Socio Economic Status [float64]
MMSE - Mini Mental State Examination [float64]
CDR - Clinical Dementia Rating [float64]
eTIV - Estimated Total Intracranial Volume [int64]
nWBV - Normalize Whole Brain Volume [float64]
ASF - Atlas Scaling Factor [float64]


#### Analiza i značenje podataka:

U ispisu iznad možemo vidjeti prvih pet redaka dataseta koji obrađujem.
objašnjenje:


### 2. Prikaz tipova podataka i deskriptivne statistike

In [42]:
m, n = df.shape
print(f'Broj stupaca dataseta: {n}')
print(f'Broj redaka dataseta: {m}')

Broj stupaca dataseta: 15
Broj redaka dataseta: 373


In [32]:
df.describe()

,Visit,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,373.000000,373.000000,354.000000,371.000000,373.000000,373.000000,373.000000,373.000000
mean,1.882038,595.104558,77.013405,14.597855,2.460452,27.342318,0.290885,1488.128686,0.729568,1.195461
std,0.922843,635.485118,7.640957,2.876339,1.134005,3.683244,0.374557,176.139286,0.037135,0.138092
min,1.000000,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,0.644000,0.876000
25%,1.000000,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,0.700000,1.099000
50%,2.000000,552.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,0.729000,1.194000
75%,2.000000,873.000000,82.000000,16.000000,3.000000,30.000000,0.500000,1597.000000,0.756000,1.293000
max,5.000000,2639.000000,98.000000,23.000000,5.000000,30.000000,2.000000,2004.000000,0.837000,1.587000


**count**: Broj neprekidnih (non-null) vrijednosti u svakom stupcu.  
**mean**: Srednja vrijednost, aritmetička sredina svih vrijednosti u stupcu.  
**std**: Standardna devijacija, mjera raspršenosti podataka koja pokazuje koliko su vrijednosti razdvojene od srednje vrijednosti.  
**min**: Minimum, najmanja vrijednost u stupcu.  
**25%**: Prvi kvartil (25. percentil) - vrijednost ispod koje se nalazi 25% podataka.  
**50%**: Drugi kvartil (50. percentil) - medijan, srednja vrijednost kad su podaci sortirani.  
**75%**: Treći kvartil (75. percentil) - vrijednost ispod koje se nalazi 75% podataka.  
**max**: Maksimum, najveća vrijednost u stupcu.  

Proučavajući ovu tablicu moćemo uočiti da za SES i MMSE nedostaje nekoliko zapisa.

### 3. Provjera nedostajućih i stršećih vrijednosti

In [43]:
df.isnull().sum()

Subject ID     0
MRI ID         0
Group          0
Visit          0
MR Delay       0
M/F            0
Hand           0
Age            0
EDUC           0
SES           19
MMSE           2
CDR            0
eTIV           0
nWBV           0
ASF            0
dtype: int64